In [70]:
# # read in the 3 txt files and combine into 1
# import glob
# import os

# # os.chdir('sample_data')
# read_files = glob.glob("sample_data/*.txt")

# with open("sample_data/combined.txt", "wb") as outfile:
#     for f in read_files:
#         with open(f, "rb") as infile:
#             outfile.write(infile.read())

In [71]:
# import the dataset
import pandas as pd
import numpy as np
import os

# Directory path
directory_path = 'C:\\Users\\azaan\\OneDrive\\Documents\\GitHub\\cs410_LLM_project\\sample_data'

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['Filename', 'Content'])

# Iterate over each file in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory_path, filename)
        
        # Read the content of the text file
        with open(file_path, 'r') as file:
            content = file.read()
        
        # Append a row to the DataFrame
        df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)

# Display the resulting DataFrame
print(df)

  Filename                                            Content
0  7_1.txt  0:00\n[SOUND] In this lecture we give an overv...
1  7_2.txt  0:00\n[SOUND] So, looking at the text mining p...
2  7_3.txt  0:00\n[SOUND]\nPlay video starting at ::9 and ...


C:\Users\azaan\AppData\Local\Temp\ipykernel_32240\3844069920.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)
C:\Users\azaan\AppData\Local\Temp\ipykernel_32240\3844069920.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)
C:\Users\azaan\AppData\Local\Temp\ipykernel_32240\3844069920.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Filename': filename, 'Content': content}, ignore_index=True)


In [72]:
# clean up words in dataset -- this includes removing stopwords
import regex as re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, words, brown

nltk.download("stopwords")
nltk.download("words")
nltk.download("brown")

lemmer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# initialize dictionary
global_dictionary  = set(words.words()) | set(brown.words())
global_dictionary = {word.lower() for word in global_dictionary}
remove_words = list(stop_words) # might need to use word_tokenize
remove_words.extend(['Play', 'video', 'starting', 'at', '::', 'follow', 'transcript']) # add the common words that's include d in transcript

# Now start actually cleaning the text
def clean_text(text):
    text = text.lower() # lowercase
    text = text.replace('\n', ' ') # remove newline indicator
    text = re.sub(r'[^a-zA-Z\s]', '', text) # case
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'http\S+|www\S+', '', text) # website
    text = re.sub(r'(\b\w+\b)(?: \1)+', r'\1', text) # remove duplicate next word after space
    text = re.sub(r'\b(?![aI]\b)\w\b', '', text)

    return text

# Remove stopwords and only keep words in dictionary
def remove_terms(text):
    text = clean_text(text)
    words = text.split()
    filtered_words = [word for word in words if word not in remove_words] # remove stopwords
    filtered_words = [word for word in filtered_words if word in global_dictionary] # remove if not in global dictionary
    return " ".join(filtered_words)

df['Content'] = df['Content'].apply(remove_terms)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\azaan\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [73]:
df['Content'][0]

'sound lecture give overview text mining analytics play first lets define term text mining term text analytics title course called text mining analytics play two terms text mining text analytics actually roughly play really going really distinguish going use interchangeably reason chosen use terms title also subtle difference look two phrases literally play mining emphasizes process gives us error rate medical view problem analytics hand emphasizes result play problem mind going look text data help us solve problem play said treat two terms roughly play think literature probably find going really distinguish course play text mining text analytics mean want turn text data high quality information actionable knowledge play cases play problem dealing lot text data hope turn text data something useful us raw text data play distinguish two different results one information actionable knowledge play sometimes boundary two clear play also want say little bit play two different angles result t

In [74]:
# Create bigrams and trigrams from data

# Function to filter bigrams or trigrams
def ngram_filter(ngram):
    tags = nltk.pos_tag(ngram)
    if not all(tag[1] in ['JJ', 'NN'] for tag in tags):
        return False
    if any(word in stop_words for word in ngram):
        return False
    if 'n' in ngram or 't' in ngram:
        return False
    if 'PRON' in ngram:
        return False
    return True

# Function to find top ngrams
def find_top_ngrams(texts, ngram_measures, min_freq=50, min_pmi=5, top_k=100):
    finder = nltk.collocations.BigramCollocationFinder.from_documents(texts)
    finder.apply_freq_filter(min_freq)
    ngram_scores = finder.score_ngrams(ngram_measures.pmi)
    filtered_ngrams = [ngram for ngram, pmi in ngram_scores if ngram_filter(ngram) and pmi > min_pmi]
    return [' '.join(ngram) for ngram in filtered_ngrams][:top_k]

bigram_measures = nltk.collocations.BigramAssocMeasures()
bigrams = find_top_ngrams([text.split() for text in df['Content']], bigram_measures)
trigram_measures = nltk.collocations.TrigramAssocMeasures()
trigrams = find_top_ngrams([text.split() for text in df['Content']], trigram_measures)

# Function to replace ngrams in text
def replace_ngrams(text):
    for gram in trigrams:
        text = text.replace(gram, '_'.join(gram.split()))
    for gram in bigrams:
        text = text.replace(gram, '_'.join(gram.split()))
    return text

# Apply ngram replacements to the text
df['Grams'] = df['Content'].map(replace_ngrams)


In [75]:
# Tokenize reviews + remove stop words + filter only nouns
def tokenize_and_filter(text):
    words = nltk.word_tokenize(text)
    words = [word.lower() for word in words if word.lower() not in stop_words and len(word) > 2]
    pos_comment = nltk.pos_tag(words)
    filtered = [word[0] for word in pos_comment if word[1] in ['NN']]
    return filtered

df['Grams'] = df['Grams'].map(tokenize_and_filter)

In [76]:
# some pytorch  training

import torch
randint = torch.randint(0, 10000, (6,1))
tensor = torch.tensor([[5, 8, 7],  [4, 8, 6], [9, 8, 4]])
zeros = torch.zeros([6,6])
ones = torch.ones([6,6])
arange = torch.arange(0, 10000)
linspace = torch.linspace(0, 10000, steps=100)
logspace = torch.logspace(-5000, 5000, steps=1000)
eye = torch.eye(6)
empty_like = torch.empty_like(eye)

randint
tensor
zeros
ones
arange
linspace
logspace
eye
empty_like

tensor([[ 1.9478e-43,  0.0000e+00,  1.9944e-07,  7.2167e-43,  2.6344e-43,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  5.9954e-36,  1.5165e-36,  8.9732e+11,
         -3.9475e-42],
        [ 0.0000e+00,  0.0000e+00,         nan,  0.0000e+00,  8.5479e-44,
          5.6052e-45],
        [ 3.5366e-04,  7.2167e-43,  1.7937e-43,  0.0000e+00,  9.5316e+11,
         -4.3062e-42],
        [ 2.6485e-43,  0.0000e+00,  3.1475e-06,  7.2167e-43,  3.5873e-43,
          0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.5165e-36,  1.5797e-37,  9.5745e+11,
         -4.7406e-42]])

In [77]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# print(device)
import torch.nn as nn
import  torch.nn.functional as F

# more pytorch fucntions
probabilities = torch.tensor([0.4, 0.6, 0.1])
samples = torch.multinomial(probabilities, num_samples=7, replacement=True)
ones = torch.ones([6,6])
tril = torch.tril(ones)
triu = torch.triu(ones)
zeros = torch.zeros(6, 6)
masked_fill = zeros.masked_fill(tril == 1, float(8))
exp = torch.exp(masked_fill)
transposed = tril.transpose(0, 1)
stack = torch.stack([tril, triu, zeros])
sample = torch.tensor([10., 10., 10.])
linear = nn.Linear(3, 3, bias=False)
sample_linear = linear(sample)
sample_softmax = F.softmax(sample)

samples
ones
tril
triu
zeros
masked_fill
exp
transposed
stack
sample
linear
sample_linear
sample_softmax

C:\Users\azaan\AppData\Local\Temp\ipykernel_32240\3418477580.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sample_softmax = F.softmax(sample)


tensor([0.3333, 0.3333, 0.3333])

In [118]:
# now I will make embeddings for my words, let's see if it works
from torch.nn.utils.rnn import pad_sequence

results = set()
df['Grams'].apply(results.update)
# print(len(results))

# Create a vocabulary dictionary
word_to_index = {word: idx for idx, word in enumerate(results)}

# Convert words to indices in your DataFrame
# AKA Encode these
df['Grams_indices'] = df['Grams'].apply(lambda x: [word_to_index[word] for word in x])

# Create a reverse dictionary
index_to_word = {idx: word for word, idx in word_to_index.items()}

# Function to convert indices back to words
def indices_to_words(indices):
    return [index_to_word[idx] for idx in indices]

# # Apply the function to the 'Grams_indices' column
# Aka Decode these grams
# df['Decoded_Grams'] = df['Grams_indices'].apply(indices_to_words)

# Pad sequences to a specified length (e.g., maxlen)
maxlen = 50  # You can adjust this based on your data
padded_indices = pad_sequence([torch.LongTensor(seq) for seq in df['Grams_indices']], batch_first=True, padding_value=0)

vocab_size = len(results)
embedding_dim = 11
embedding = nn.Embedding(vocab_size, embedding_dim)

# embedded_indices = torch.LongTensor(df['Grams_indices'].tolist())
embedded_grams = embedding(padded_indices)

In [114]:
# matrix multiplication examples with pytorch
a = torch.tensor([[3, 4], [5, 8], [8, 3]])
b = torch.tensor([[8, 26, 5], [34, 7, 17]])

# form matmul, can do either below
a @ b
matmul = torch.matmul(a, b)

matmul

tensor([[160, 106,  83],
        [312, 186, 161],
        [166, 229,  91]])

In [137]:
# make a batch
block_size = 8
batch_size = 4

# not sure if I should use the embedded grams for this batch
# Flatten the embedded_grams tensor
data = embedded_grams.view(-1, embedded_grams.size(-1))
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
# print(len(data))

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

x, y = get_batch('train')
print(x)
print(y)


tensor([ 41,  28, 233, 281])
tensor([[[-6.5571e-01, -3.7480e-01,  6.0739e-01, -6.6817e-01, -5.9260e-01,
          -2.7353e+00,  1.9190e-01, -8.9298e-01, -1.0818e+00, -3.4203e-01,
           9.5509e-01],
         [-5.0029e-01,  1.0881e+00,  5.7089e-01, -2.2891e-01,  8.6399e-01,
          -1.7891e+00, -7.4548e-01,  7.7717e-01, -3.3532e-01,  5.4402e-01,
           2.1459e-02],
         [-8.6783e-01, -2.1527e+00,  1.0966e+00,  4.5674e-01,  5.4685e-01,
           1.4204e+00, -3.7695e-01, -2.4782e+00, -2.1762e+00,  1.2243e-01,
           8.4708e-01],
         [-4.8467e-01, -3.6913e-01,  1.2088e+00,  2.4618e-01, -1.0662e+00,
           8.6203e-01,  7.8856e-02, -4.8550e-02, -8.2839e-01,  1.3926e-02,
           1.9957e-02],
         [-1.8060e-01, -5.9862e-01,  8.6913e-01,  1.3451e+00,  1.5062e+00,
          -1.5742e+00, -1.6150e-01, -1.6652e+00, -9.4644e-01, -1.7916e-02,
           2.1530e+00],
         [ 7.5653e-01,  2.6278e+00, -7.6779e-01, -1.3161e+00,  6.4904e-01,
           8.8990e-01,  1.

In [ ]:
# Now to make a simple bigram model
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table == nn.Embedding(vocab_size, vocab_size)


        



